<div class="alert alert-block alert-success">
    
# FIT5196 Task 1 in Assessment 1
#### Student Name: Deshui Yu      Liangjing Yang
#### Student ID: 34253599      34060871

Date: 21/08/2024


Environment: Python xxxx

Libraries used:
* re (for regular expression, installed and imported) 
* pandas (for data manipulation) 
    
</div>

<div class="alert alert-block alert-danger">
    
## Table of Contents

</div>    

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Patent Files](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Defining Regular Expressions](#Reg_Exp) <br>
$\;\;\;\;$[4.2. Reading Files](#Read) <br>
$\;\;\;\;$[4.3. Whatever else](#latin) <br>
[5. Writing to CSV/JSON File](#write) <br>
$\;\;\;\;$[5.1. Verification - using the sample files](#test_xml) <br>
[6. Summary](#summary) <br>
[7. References](#Ref) <br>

-------------------------------------

<div class="alert alert-block alert-warning">

## 1.  Introduction  <a class="anchor" name="Intro"></a>
    
</div>

This assessment regards extracting data from semi-sctuctured text files. The dataset contained 500 `.txt` files which included various information about user reviews. In particular, ....

-------------------------------------

<div class="alert alert-block alert-warning">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>
 </div>

The packages to be used in this assessment are imported in the following. They are used to fulfill the following tasks:

* **re:** to define and use regular expressions
* **pandas:** ...
* ...

In [1]:
import re
import pandas
import pandas as pd
from datetime import datetime
# My assignment is running locally, and I have already downloaded the data to my local system, so I don't need to link to Google Drive.
# from google.colab import drive
# drive.mount('/content/drive')

-------------------------------------

<div class="alert alert-block alert-warning">

## 3.  Examining Raw Data <a class="anchor" name="examine"></a>

 </div>

First of all, open all the files and examine the data within them.

Having examined the file content, the following observations were made:

Examining the XLSX File:
- By opening the XLSX file, I discovered that there are sheets numbered from 0 to 14. This means I need to combine these sheets into a single DataFrame.
- By observing the different sheets, I noticed that some of them contain entire rows and columns of empty values. I think I need to remove these empty values.
- After removing the irrelevant empty rows and columns, I found that all sheets have the same content. However, the format of the date and other data does not meet the requirements of the assignment.

Examining the TXT Files:
- I have 15 files numbered from 0 to 14. The first line of each TXT file contains the label `<?xml version="1.0" encoding="UTF-8"?>`, which is a common point. I can use this to split the data.
- I also noticed that the label `</dataset><record>` can be used to split the data for each user, and this pattern is consistent across all TXT files.
- In the TXT files, each piece of data is wrapped in tags, such as userid, username, rate, pictures, etc. Additionally, the tags are quite varied; for instance, the rate data can have multiple tag forms such as `<Rate>`, `</rate>`, `<rating>`, `</Rating>`, and more. This means that if I want to extract the data, my regular expressions will need to be compatible with these different tag forms.


-------------------------------------

<div class="alert alert-block alert-warning"> 

## 4.  Loading and Parsing Files <a class="anchor" name="load"></a>

</div>

In this section, the files are parsed and processed. First of all, appropriate regular expressions are defined to extract desired information when reading the files. ....

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.1. Defining Regular Expressions <a class="anchor" name="Reg_Exp"></a>

Defining correct regular expressions is crucial in extracting desired information from the text efficiently. ...

In [2]:
user_id_expression = r"<[^>]*?ser[^>]*?>\s*(\d{21})\s*<[^>]*?/[^>]*?ser[^>]*?>"
time_expression = r"(?:<.{1,3}ime\s*/?>\s*|<.{0,2}[Dd]ate\s*/?>\s*)(\d+)(?:\s*<\s*/\s*.{1,3}ime\s*/?>|<\s*/\s*.{0,2}[Dd]ate\s*/?>)"
rate_expression = r"(?:<.{0,2}[Rr]ate>\s*)(\d+)(?:<.{0,2}[Rr]ate>)"
rateing_expression = r"(?:<.{0,2}[Rr]ating>\s*)(\d+)(?:<.{0,2}[Rr]ating>)"
text_expression = r"(?:<.{0,3}[Tt]ext>\s*)(.*?)(?:<.{0,3}[Tt]ext>)"
review_expression = r"(?:<.{0,2}[Rr]eview>\s*)(.*?)(?:<.{0,2}[Rr]eview>)"
name_expression = r"(?:<.{0,7}[Nn]ame>\s*)(.*?)(?:<.{0,7}[Nn]ame\s*>)"
response_expression = r"(?:<.{0,3}[Rr]esp.{0,4}>\s*)(.*?)(?:<.{0,3}[Rr]esp.{0,4}>)"
picture_expression = r"(?:<.{0,2}[Pp]ic.{1,5}>\s*)(.*?)(?:<.{0,2}[Pp]ic.{1,5}>)"
gmap_id_expression=r"(?:<.{0,2}[Gg]map.{1,3}>\s*)(.*?)(?:<.{0,2}[Gg]map.{1,3}>)"

In [3]:
text_path = './student_group181'
all_text_data = ""
file_names = [
    'group181_0.txt', 'group181_1.txt', 'group181_2.txt', 'group181_3.txt', 
    'group181_4.txt', 'group181_5.txt', 'group181_6.txt', 'group181_7.txt', 
    'group181_8.txt', 'group181_9.txt', 'group181_10.txt', 'group181_11.txt',
    'group181_12.txt', 'group181_13.txt', 'group181_14.txt'
]
for file_name in file_names:
    file_path = f"{text_path}/{file_name}"
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        all_text_data += content
        
split_txt = '?xml version="1.0" encoding="UTF-8"?>'
final_split= '</dataset><record>|<dataset><record>'
folder_text = all_text_data.split(split_txt)
if folder_text[0] == "":
    folder_text.pop(0)
final_split_text = []
for text in folder_text:
    slipted_info = re.split(final_split,text)
    final_split_text.extend(slipted_info)

In [4]:
user_id_list = []
gmap_id_list = []
time_list = []
date_list = []
rate_list = []
review_list = []
response_list = []
picture_list = []
name_list = []

user_id_pattern = re.compile(user_id_expression, re.DOTALL)
time_pattern = re.compile(time_expression, re.DOTALL)
rate_pattern = re.compile(rate_expression, re.DOTALL)
rateing_pattern = re.compile(rateing_expression, re.DOTALL)
text_pattern = re.compile(text_expression, re.DOTALL)
review_pattern = re.compile(review_expression, re.DOTALL)
response_pattern = re.compile(response_expression, re.DOTALL)
picture_pattern = re.compile(picture_expression, re.DOTALL)
name_pattern = re.compile(name_expression,re.DOTALL)
gmap_id_pattern = re.compile(gmap_id_expression, re.DOTALL)

for text in final_split_text:
     
     user_id_matches = user_id_pattern.findall(text)
     user_id_list.extend(user_id_matches)
     
     gmap_id_matches = gmap_id_pattern.findall(text)
     gmap_id_list.extend(gmap_id_matches)
     
     time_matches = time_pattern.findall(text)
     time_list.extend(time_matches)
     
     rate_matches = rate_pattern.findall(text)
     if not rate_matches:
        rate_matches = rateing_pattern.findall(text)
     rate_list.extend(rate_matches)
     
     review_matches = review_pattern.findall(text)
     if not review_matches:
        review_matches = text_pattern.findall(text)
     review_list.extend(review_matches)
     
     response_matches = response_pattern.findall(text)
     response_list.extend(response_matches)
     
     picture_matches = picture_pattern.findall(text)
     picture_list.extend(picture_matches)
     
     
     name_matches = name_pattern.findall(text)
     name_list.extend(name_matches)

text_datafram = pd.DataFrame({
    'user_id': user_id_list,
    'name':name_list,
    'time': time_list,
    'rating': rate_list,
    'text': review_list,
    'pics': picture_list,
    'resp': response_list,
    'gmap_id': gmap_id_list
})

text_datafram = text_datafram[['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp','gmap_id']]
text_datafram['rating'] = text_datafram['rating'].astype(float)
text_datafram['time'] = pd.to_datetime(pd.to_numeric(text_datafram['time']), unit='ms', utc=True)
text_datafram['time'] = text_datafram['time'].dt.strftime("%Y-%m-%d %H:%M:%S")

text_datafram = text_datafram.replace('None', None)

if_pic = []
if_response = []
for index, row in text_datafram.iterrows():
    if row['pics'] is None:
        if_pic.append('N')
    else:
        if_pic.append('Y')
    
    if row['resp'] is None:
        if_response.append('N')
    else:
        if_response.append('Y')
text_datafram['if_pic'] = if_pic
text_datafram['if_response'] = if_response
def extract_dimensions(text):
    if pd.isna(text):
        return "[ ]"
    dimensions = re.findall(r'w(\d{1,3})-h(\d{1,3})-k-no-p', text)
    if not dimensions:
        return "[ ]"
    return [[int(width), int(height)] for width, height in dimensions]

text_datafram['pic_dim'] = text_datafram['pics'].apply(extract_dimensions)
text_datafram


,user_id,name,time,rating,text,pics,resp,gmap_id,if_pic,if_response,pic_dim
0,103499789493991350231,Alyson Hoffman,2021-06-01 04:57:49,5.0,Booked last minute rentals here over Memorial ...,None,None,0x808e6a7f6b666d4f:0xfd9214abe1df3a4d,N,N,[ ]
1,105404622065259965715,OnlyOneEarth,2021-03-21 07:59:56,5.0,"When I went into Epicenter Cycling, I was look...",[{'url': ['https://lh5.googleusercontent.com/p...,"{'time': 1616349255531, 'text': ""We're so happ...",0x808e6a7f6b666d4f:0xfd9214abe1df3a4d,Y,Y,"[[150, 150], [150, 150]]"
2,118347770680889723845,matthew thomson,2021-04-11 15:44:41,5.0,Amazing selection of Trek bikes! We were told ...,None,"{'time': 1622404153725, 'text': ""Thanks for ta...",0x808e6a7f6b666d4f:0xfd9214abe1df3a4d,N,Y,[ ]
3,109541606733835908756,Becky B,2021-03-15 15:42:06,5.0,"Absolutely flawless experience ordering, sizin...",[{'url': ['https://lh5.googleusercontent.com/p...,"{'time': 1616264680468, 'text': 'Thank you so ...",0x808e6a7f6b666d4f:0xfd9214abe1df3a4d,Y,Y,"[[150, 150]]"
4,108792737827746934756,Tiffany Klitzke,2021-02-02 18:00:03,5.0,All of the employees were so friendly and help...,None,"{'time': 1614824613439, 'text': ""Thank you so ...",0x808e6a7f6b666d4f:0xfd9214abe1df3a4d,N,Y,[ ]
...,...,...,...,...,...,...,...,...,...,...,...
44692,112497754617102655494,Bernard Janov,2020-05-21 18:56:48,5.0,None,None,None,0x80dbfe36c3b7a2c7:0x8e10768a0238d2b9,N,N,[ ]
44693,112173222860157032040,Ernest Ikei,2019-06-07 00:44:22,5.0,None,None,None,0x80dbfe36c3b7a2c7:0x8e10768a0238d2b9,N,N,[ ]
44694,105511521957721541111,Randall Rolfe,2020-09-01 23:44:36,5.0,None,None,"{'time': 1613606949222, 'text': 'Thank you, Ra...",0x80dbfe36c3b7a2c7:0x8e10768a0238d2b9,N,Y,[ ]
44695,104981526115446298575,Donna Nagel,2020-10-15 19:59:21,5.0,None,None,"{'time': 1613605845348, 'text': 'Thank you for...",0x80dbfe36c3b7a2c7:0x8e10768a0238d2b9,N,Y,[ ]


In [5]:
from datetime import datetime

xls_path = "./student_group181/group181.xlsx"

xls = pd.ExcelFile(xls_path)
all_sheets = pd.read_excel(xls_path, sheet_name=None)
cleaned_sheets = []
for sheet_name, df in all_sheets.items():
    df = df.dropna(how='all')
    # 去除空的列
    df = df.dropna(axis=1, how='all')
    # 仅在DataFrame有数据时，添加到清理后的列表中
    if not df.empty:
        cleaned_sheets.append(df)

# 将所有清理后的DataFrame合并为一个
xls_data = pd.concat(cleaned_sheets, ignore_index=True)
xls_data = xls_data[['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp','gmap_id']]
#转化时间格式
xls_data['time'] = pd.to_datetime(xls_data['time'], unit ='ms', utc = True)
#将time列格式化为字符串格式
xls_data['time'] = xls_data['time'].dt.strftime("%Y-%m-%d %H:%M:%S")

xls_data['pics'] = xls_data['pics'].where(pd.notna(xls_data['pics']), None)
xls_data['resp'] = xls_data['resp'].where(pd.notna(xls_data['resp']), None)

if_pic_xls = []
if_response_xls = []
for index, row in xls_data.iterrows():
    # 如果 'pics' 列为 None，添加 'N'，否则添加 'Y'
    if row['pics'] is None:
        if_pic_xls.append('N')
    else:
        if_pic_xls.append('Y')
    
    # 如果 'resp' 列为 None，添加 'N'，否则添加 'Y'
    if row['resp'] is None:
        if_response_xls.append('N')
    else:
        if_response_xls.append('Y')



xls_data['if_pic'] = if_pic_xls
xls_data['if_response'] = if_response_xls

def extract_dimensions(text):
    if pd.isna(text):
        return "[ ]"
    dimensions = re.findall(r'w(\d{1,3})-h(\d{1,3})-k-no-p', text)
    if not dimensions:
        return "[ ]"
    return [[int(width), int(height)] for width, height in dimensions]

xls_data['pic_dim'] = xls_data['pics'].apply(extract_dimensions)

#xls_data

combined_data = pd.concat([xls_data, text_datafram], axis=0)
#print(combined_data.info())
#combined_data = combined_data.apply(lambda x: x.str.encode('utf-8').str.decode('utf-8') if x.dtype == 'object' else x)
combined_data['text'] = combined_data['text'].apply(
    lambda x: re.sub(
        r"[\U0001F600-\U0001F64F"  
        r"\U0001F300-\U0001F5FF"  
        r"\U0001F680-\U0001F6FF"  
        r"\U0001F1E0-\U0001F1FF"  
        r"\U00002702-\U000027B0"  
        r"\U000024C2-\U0001F251"  
        r"]+", '', x) if isinstance(x, str) else x
)

combined_data['text'] = combined_data['text'].str.lower()

In [6]:

grouped = combined_data.groupby('gmap_id').agg(
    review_count=('gmap_id', 'size'),
    review_text_count=('text', lambda x: x.notna().sum()),
    response_count=('resp', lambda x: x.notna().sum())
).reset_index()

grouped.to_csv('task1_group181.csv', index=False)

These patterns are used in the next step when reading the files.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.2. Reading Files <a class="anchor" name="Read"></a>

In this step, all files are read and parsed. 

Let's take a look at the first ten elements of the lists generated. We can see that ids, reviews,etc. are parsed and stored correctly.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.3. Whatever else <a class="anchor" name="latin"></a>

the rest of your methodology

-------------------------------------

<div class="alert alert-block alert-warning"> 

## 5.  Writing to JSON File <a class="anchor" name="write"></a>

</div>

....

-------------------------------------

<div class="alert alert-block alert-info">
    
### 5.1. Verification of the Generated JSON File <a class="anchor" name="test_xml"></a>

<div class="alert alert-block alert-warning"> 

## 6. Summary <a class="anchor" name="summary"></a>

</div>

....

-------------------------------------

....

-------------------------------------

<div class="alert alert-block alert-warning"> 

## 7. References <a class="anchor" name="Ref"></a>

</div>



[1]<a class="anchor" name="ref-2"></a> Why do I need to add DOTALL to python regular expression to match new line in raw string, https://stackoverflow.com/questions/22610247, Accessed 30/08/2022.

....


## --------------------------------------------------------------------------------------------------------------------------